In [2]:
import sqlalchemy as sql
import pandas as pd
import numpy as np
import csv
import json
from jsonpath_ng.ext import parse 

In [3]:
awards_filepath = "src/files/220k_awards_by_directors.csv"
movies_filepath     = "src/files/AllMoviesDetailsCleaned.csv"
cast_filepath       = "src/files/AllMoviesCastingRaw.csv"

awards_df = pd.read_csv(awards_filepath, delimiter=",", na_values='none')
movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')
cast_df = pd.read_csv(cast_filepath, delimiter=";", na_values='none')




C:\Users\rober\AppData\Local\Temp\ipykernel_344\4140812894.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(movies_filepath, delimiter=";", na_values='none')


In [4]:
awards_df

,director_name,ceremony,year,category,outcome,original_language
0,Aki Kaurismäki,ACCEC Awards,2004.0,ACCEC Award,Won,fi
1,Aki Kaurismäki,Alliance of Women Film Journalists,2012.0,EDA Award,Nominated,fi
2,Aki Kaurismäki,"Amanda Awards, Norway",1999.0,Amanda,Nominated,fi
3,Aki Kaurismäki,Argentinean Film Critics Association Awards,2013.0,Silver Condor,Nominated,fi
4,Aki Kaurismäki,Argentinean Film Critics Association Awards,2004.0,Silver Condor,Nominated,fi
...,...,...,...,...,...,...
225670,Courtney Hoffman,Satellite Awards,2016.0,Satellite Award,Nominated,en
225671,Fanni Metelius,Berlin International Film Festival,2012.0,Crystal Bear,Nominated,sv
225672,Fanni Metelius,Guldbagge Awards,2015.0,Guldbagge,Nominated,sv
225673,Fanni Metelius,Uppsala International Short Film Festival,2012.0,Honorable Mention,Won,sv


In [6]:
movies_df.head()

,id,budget,genres,imdb_id,original_language,original_title,overview,popularity,production_companies,production_countries,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,production_companies_number,production_countries_number,spoken_languages_number
0,2,0,Drama|Crime,tt0094675,fi,Ariel,Taisto Kasurinen is a Finnish coal miner whose...,0.823904,Villealfa Filmproduction Oy,Finland,...,69.0,suomi,Released,NaN,Ariel,7.1,40,2,1,2
1,3,0,Drama|Comedy,tt0092149,fi,Varjoja paratiisissa,"An episode in the life of Nikander, a garbage ...",0.47445,Villealfa Filmproduction Oy,Finland,...,76.0,English,Released,NaN,Shadows in Paradise,7.0,32,1,1,3
2,5,4000000,Crime|Comedy,tt0113101,en,Four Rooms,It's Ted the Bellhop's first night on the job....,1.698,Miramax Films,United States of America,...,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,485,2,1,1
3,6,0,Action|Thriller|Crime,tt0107286,en,Judgment Night,"While racing to a boxing match, Frank, Mike, J...",1.32287,Universal Pictures,Japan,...,110.0,English,Released,Don't move. Don't whisper. Don't even breathe.,Judgment Night,6.5,69,3,2,1
4,8,42000,Documentary,tt0825671,en,Life in Loops (A Megacities RMX),Timo Novotny labels his new project an experim...,0.054716,inLoops,Austria,...,80.0,English,Released,A Megacities remix.,Life in Loops (A Megacities RMX),6.4,4,1,1,5


In [5]:
cast_df.head()

,id,actor1_name,actor1_gender,actor2_name,actor2_gender,actor3_name,actor3_gender,actor4_name,actor4_gender,actor5_name,actor5_gender,actor_number,director_name,director_gender,director_number,producer_name,producer_number,screeplay_name,editor_name
0,2,Turo Pajala,0,Susanna Haavisto,0.0,Matti Pellonpää,2,Eetu Hilkamo,0,NaN,0,4,Aki Kaurismäki,0.0,1,NaN,0,Aki Kaurismäki,Raija Talvio
1,3,Matti Pellonpää,2,Kati Outinen,1.0,Sakari Kuosmanen,2,Esko Nikkari,2,Kylli Köngäs,0,7,Aki Kaurismäki,0.0,1,Mika Kaurismäki,1,Aki Kaurismäki,Raija Talvio
2,5,Tim Roth,2,Antonio Banderas,2.0,Jennifer Beals,1,Madonna,1,Marisa Tomei,1,24,Allison Anders,1.0,4,Lawrence Bender,1,NaN,Margaret Goodspeed
3,6,Emilio Estevez,2,Cuba Gooding Jr.,2.0,Denis Leary,2,Jeremy Piven,2,Peter Greene,2,15,Stephen Hopkins,2.0,1,Gene Levy,1,Lewis Colick,Tim Wellburn
4,8,NaN,0,NaN,0.0,NaN,0,NaN,0,NaN,0,0,Timo Novotny,0.0,1,Timo Novotny,2,Michael Glawogger,Timo Novotny


In [6]:
def processMovies(df, movie_file, company_file, company_produced_file):
    movie_header = ["movieID:ID(Movie-ID)", "title", "genres:string[]", "overview", "languages", "rating:float", "votes:int", "budget:int",":LABEL"]
    cur_df = df
    cur_df["label"]= "Movie"

    cur_df['spoken_languages'].replace("No Language", "",inplace=True)
    
    # write movies file 
    cur_df[["id", "title", "genres", "overview", "original_language",  "vote_average", "vote_count", "budget", "label"]].to_csv(movie_file, sep=";", index=False,
                                                                                                  lineterminator="", encoding="utf-8", header=movie_header)
    
    

In [7]:
processMovies(movies_df, "movie.csv")

In [10]:
def processAwards(df, node_file, relation_file):
    awards_header = ["name:ID(Award-ID)", ":LABEL"]
    relation_header = [":START_ID(Person-ID)",":TYPE", "years:int[]", ":END_ID(Award-ID)"]
    cur_df = df
    cur_df['name_clean'] = cur_df['director_name'].str.strip()
    cur_df2 = cur_df[['name_clean','category','outcome','year']].groupby(['name_clean','category','outcome'])['year'].unique().reset_index()
    cur_df2['yearstr'] = cur_df2.year.apply(lambda x : "|".join("%4.0f" % y for y in x))
    
    cur_df3 = cur_df2[['name_clean','outcome','yearstr','category']]
    awards = []
    
    # get nodes
    for award in df["category"].unique():
        awards.append([award, "Award"])
            
    # write nodes file 
    with open(node_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(awards_header)
        for row in awards:
            writer.writerow(row)
    # wirte relation file 
    cur_df3.to_csv(relation_file, sep=";", index=False,
                  lineterminator="", encoding="utf-8", header=relation_header)
    
    return cur_df3['name_clean'].unique()

In [11]:
directors = processAwards(awards_df, "award.csv", "awarded.csv")

In [18]:
def processPerson(row, person_data, rel_data, columns, role, extra_label):
    # Must be within the bounds
    if len(columns) < 1 or len(columns) > 2:
        return
    
    item = row[columns]
    
    # Get person info
    name = item[0]
    
    # Skip if NaN
    if name is np.nan:
        return
    
    # Initialize list with directors 
    
    if len(columns) == 2:
        if item[1] == 1:
            gender = "f"
        elif item[1] == 2:
            gender = "m"
        else:
            gender = "na"
    else:
        gender = "na"
        
    # Populate person info
    if name in person_data:
        item_new = person_data[name]
        if gender == "f" or gender == "m":
            item_new[1] = gender
        if extra_label not in item_new[2]:
            item_new[2] = item_new[2] + "|" + extra_label
    else:
        item_new = [name, gender, "Person" + "|" + extra_label]
            
    # Movie relationship
    movieID = row["id"]
    person_data[name] = item_new
    rel_data.append([name,role,movieID])
    
def processCast(df, directors, person_file, roles_file):
    
    persons_header = ["name:ID(Person-ID)", "gender", ":LABEL"]
    roles_header = [":START_ID(Person-ID)",":TYPE", ":END_ID(Movie-ID)"]
    
    col_actor1 = ['actor1_name', 'actor1_gender']
    col_actor2 = ['actor2_name', 'actor2_gender']
    col_actor3 = ['actor3_name', 'actor3_gender'] 
    col_actor4 = ['actor4_name', 'actor4_gender']
    col_actor5 = ['actor5_name', 'actor5_gender']
    col_director = ['director_name', 'director_gender']
    col_producer = ['producer_name']
    col_screenplay = ['screeplay_name']
    col_editor = ['editor_name']
    
    
    person_data = {}
    roles_data = []
    
    #Initialize pperson data with directors we extracted
    for director in directors:
        person_data[director] = [director, "na", "Person|Director"]
        
    for i, row in df.iterrows():
        processPerson(row, person_data, roles_data, col_actor1, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor2, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor3, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor4, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_actor5, "ACTED_IN", "Actor")
        processPerson(row, person_data, roles_data, col_director, "DIRECTED", "Director")
        processPerson(row, person_data, roles_data, col_producer, "PRODUCED", "Producer")
        processPerson(row, person_data, roles_data, col_screenplay, "SCREENPLAYED", "Screenwriter")
        
    # write nodes file 
    with open(person_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow(persons_header)
        for row in person_data.values():
            writer.writerow(row)
    # wirte relation file 
    with open(roles_file, 'w',  newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(roles_header)
        for row in roles_data:
            writer.writerow(row)

In [19]:
processCast(cast_df, directors, "person.csv", "role.csv")

In [ ]:
# run neo4j-admin to load data to neo4j 
# .\bin\neo4j-admin.bat import --force --multiline-fields=true --delimiter=";" --array-delimiter="|" --database=final --nodes=.\import\award.csv --nodes=.\import\movie.csv --nodes=.\import\person.csv --relationships=.\import\awarded.csv --relationships=.\import\role.csv